In [3]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC
from fgz.training.fgz_trainer import FGZTrainer
import torch

In [4]:
torch.cuda.empty_cache()  # fix memory leaks

In [5]:
minerl_env = gym.make('MineRLBasaltFindCave-v0')
agent = get_agent()
dynamics_env = get_dynamics_environment(minerl_env)

In [7]:
dynamics_env

In [ ]:
fmc = FMC(dynamics_env)
trainer = FGZTrainer(minerl_env, agent, fmc, unroll_steps=8)